In [17]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json
import requests

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended,process_datesIT

In [18]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [19]:
independents = urls[[name for name in urls.index if name[0] == 'G']]
dependents = urls[[name for name in urls.index if name[0] != 'G' and name[0] != 'U']]
unused = urls[[name for name in urls.index if name[0] == 'U']]

In [20]:
! rm ../../Finances.db
! rm schema.sql

In [21]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [22]:
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()
schemas = open("schema.sql", "a+")

In [23]:
%%time
for name, url in independents.iteritems():
    print(name,url)
    tab = get_appended(url,apikey,name)
    a = pd.io.sql.get_schema(tab,name)
    c.execute('drop table if exists '+ name)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(name,con=conn,index=False,if_exists='append')

G1_CV_SYMBOLS https://financialmodelingprep.com/api/v3/stock/list?apikey=
G1_CV_ETFS https://financialmodelingprep.com/api/v3/etf/list?apikey=
G1_CV_TRADABLE https://financialmodelingprep.com/api/v3/available-traded/list?apikey=
G1_CV_DELISTED https://financialmodelingprep.com/api/v3/delisted-companies?limit=1000000000&apikey=
G1_CAL_EARNINGSCALENDAR_UPC https://financialmodelingprep.com/api/v3/earning_calendar?apikey=
G1_INST_CIKLIST https://financialmodelingprep.com/api/v3/cik_list?apikey=
G1_MI_MAJORINDEXES https://financialmodelingprep.com/api/v3/quotes/index?apikey=
G1_MI_SP500LIST https://financialmodelingprep.com/api/v3/sp500_constituent?apikey=
G1_MI_SP500HIST https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey=
G1_MI_NASDAQLIST https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=
G1_MI_NASDAQHISTLIST https://financialmodelingprep.com/api/v3/historical/nasdaq_constituent?apikey=
G1_MI_DOWJLIST https://financialmodelingprep.com/api/v3/dowj

In [15]:
#setup database
###FULL lists for full set processing
stocks = list(pd.io.sql.read_sql('select symbol from G1_CV_SYMBOLS;',con=conn).symbol)
etfs  = list(pd.io.sql.read_sql('select symbol from G1_ETF_SYMBOLS;',con=conn).symbol)
etfs2  = list(pd.io.sql.read_sql('select symbol from G1_CV_ETFS;',con=conn).symbol)
indexes = list(pd.io.sql.read_sql('select symbol from G1_MI_MAJORINDEXES;',con=conn).symbol)
ciks = ['000'+str(c) for c in list(pd.io.sql.read_sql('select cik from G1_INST_CIKLIST',con=conn).cik)]
funds = list(pd.io.sql.read_sql('select symbol from G1_MF_SYMBOLS;',con=conn).symbol)
commods = list(pd.io.sql.read_sql('select symbol from G1_COM_SYMBOLS;',con=conn).symbol)
##Light cleanup, nothing perfect –– could set this up using sql. 

#merge etf lists
etfs_c = list(set(etfs).union(set(etfs2)))
#leave stocks only 
stocks_c = list(set(stocks) - set(etfs_c + indexes + ciks + funds + commods))

In [9]:
#setup subset of tickers
dow_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_DOWJLIST;',con=conn).symbol)
nasdaq_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_NASDAQLIST;',con=conn).symbol)
sp500_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_SP500LIST;',con=conn).symbol)

In [10]:
#define subset to work with
stocksub = dow_tickers
#stocksub = sp500_tickers
indexsub = ['^IXIC','^GSPC','^DJI']
#indexsub = ['^IXIC','^GSPC','^DJI','^VIX','^RUA','^FTSE','^NDX','^N225','GDAXI']

In [11]:
#get top US etfs proportionate to stock list
num_etfs = int(np.ceil(len(stocksub)/10))
profile = get_appended(dependents['P1b1000_CV_PROFILE'],apikey,'P1b1000_CV_PROFILE',tickers = etfs_c)
etfsub = list(profile[profile.country == 'US'].sort_values(by='volAvg',ascending=False).symbol[:num_etfs+1])
tickersub = stocksub + etfsub

In [24]:
#get mutual fund data
fundtable = get_appended(dependents['T1_INST_MUTFUNDHOLDERS'],apikey,'T1_INST_MUTFUNDHOLDERS',tickers=tickersub,verbose=False)
fundsub = list(fundtable.holder.value_counts().index)

NameError: name 'tickersub' is not defined

In [14]:
mfs = pd.io.sql.read_sql('select * from G1_MF_SYMBOLS;',con=conn)
fundsub = list(np.random.choice(list(mfs[mfs.name.isin(fundsub)].symbol),int(np.ceil(len(stocksub)/3))))

In [19]:
#establish subset of commods
#commodsub = commods[:3]
commodsub = commods

In [16]:
#get ciks of institutional holders for ciksub
instable = get_appended(dependents['T1_INST_HOLDERS'],apikey,'T1_INST_HOLDERS',tickers=tickersub)
instsub = list(instable.holder.value_counts().index)
instsub = list(np.random.choice(instsub,500)) #account for many missing inst-cik mappings

In [17]:
#retrieve ciks
ciklist = pd.io.sql.read_sql('select * from G1_INST_CIKLIST',con=conn)
ciksub = list(ciklist[ciklist.name.apply(lambda x: x.lower()).isin([i.lower() for i in instsub])].cik)
ciksub = ['000'+str(c) for c in (ciksub)]
ciksub = [c for c in ciksub if len(c)==10]

In [20]:
#dictionary – nothing commodity, crypto, etc. specific
d = {'H':stocksub + etfsub + commodsub + fundsub,
     'P':stocksub + etfsub + fundsub,
     'T':stocksub + etfsub,
     'S':stocksub,
     'E':etfsub,
     'C':commodsub,
     'F':ciksub,
     'I':indexsub,
     'M':fundsub}

In [21]:
#dependentsub = dependents.iloc[43:] #for testing specific urls

CV_PROFILE
Running Batch: MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALXN,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,AOS,APA,AAPL,AMAT,APTV,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BK,BAX,BDX,BRK.B,BBY,BIO,BIIB,BLK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BF.B,CHRW,COG,CDNS,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CERN,CF,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,COO,CPRT,GLW,CTVA,COST,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,EVRG,ES,RE,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FE,FRC,FISV,FLT,FLIR,FLS,FMC,F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GD,GE,GIS,GM,GPC,GILD,GL,GPN,GS,GWW,HAL,HBI,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HFC,HOLX,HD,HON,HRL,H

<timed exec>:20: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
/Users/verbeck/opt/anaconda3/envs/ide/lib/python3.9/site-packages/pandas/core/generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


CV_DCF_YR
index 0 is out of bounds for axis 0 with size 0  for ticker CARR, continuing.....
index 0 is out of bounds for axis 0 with size 0  for ticker LUMN, continuing.....
index 0 is out of bounds for axis 0 with size 0  for ticker OTIS, continuing.....
index 0 is out of bounds for axis 0 with size 0  for ticker VTRS, continuing.....
index 0 is out of bounds for axis 0 with size 0  for ticker VNT, continuing.....


<timed exec>:20: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
/Users/verbeck/opt/anaconda3/envs/ide/lib/python3.9/site-packages/pandas/core/generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


CV_DCF_QTR
index 0 is out of bounds for axis 0 with size 0  for ticker LUMN, continuing.....
index 0 is out of bounds for axis 0 with size 0  for ticker VTRS, continuing.....


<timed exec>:20: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
/Users/verbeck/opt/anaconda3/envs/ide/lib/python3.9/site-packages/pandas/core/generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


CV_DCF_DAY
[Errno 54] Connection reset by peer  for ticker CMI, continuing.....
[Errno 54] Connection reset by peer  for ticker IRM, continuing.....
CV_MARKETCAP
CV_MARKETCAP_DAY
CV_NEWS
Running Batch: MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALXN,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,AOS,APA,AAPL,AMAT,APTV,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BK,BAX,BDX,BRK.B,BBY,BIO,BIIB,BLK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BF.B,CHRW,COG,CDNS,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CERN,CF,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,COO,CPRT,GLW,CTVA,COST,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,EVRG,ES,RE,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,F